In [1]:

import argparse

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

def create_model():
    # 8bit quant
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/xgen-7b-8k-inst", trust_remote_code=True, cache_dir = "./cache/", device_map="auto")
    model = AutoModelForCausalLM.from_pretrained("Salesforce/xgen-7b-8k-inst", torch_dtype=torch.bfloat16, cache_dir = "./cache/", use_cache=True, device_map="auto")
    # model = model.half()
    return model, tokenizer

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

def create_dataset():
    val_dataset = load_dataset(
        "EdinburghNLP/xsum", split="validation", cache_dir="./cache/"
    )
    test_dataset = load_dataset(
        "EdinburghNLP/xsum", split="test", cache_dir="./cache/"
    )
    return val_dataset, test_dataset

In [4]:
model, tokenizer = create_model()

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.13s/it]


In [5]:
val_dataset, test_dataset = create_dataset()

In [6]:
length = 0.0
small = 0.0
with torch.no_grad():
        for i, data in enumerate(test_dataset):
            article = data["document"]
            if len(article) < 7000:
                small += 1
                
print(small)
print(len(test_dataset))



11120.0
11334


In [7]:
# model = model.to("cuda:0")
# article = "I am from Japan and My father is a farmer. We live a normal life. My name is John. I am 20 years old. I am a student."
prompt = f"### Human: Please summarize the following article.\n\n{article}.\n###"
print(prompt)

### Human: Please summarize the following article.

Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, families would have to ration their food and had little communication in their homes.
Luxuries like chocolate and fruit were very difficult to find and families had to grow their own food to survive.
Watch Martin's report to find out more..
###


In [8]:
inputs = tokenizer(prompt, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[21017,  5524,    25,  4222, 35743,   262,  1708,  2708,    13,   198,
           198, 13152, 16388,   318, 24222,   319,   534,  3335,   198,    23,
          1737,  1853,  4586,  6153,   379,   838,    25,  2078, 44992,   198,
          7191,   262,  1175,    11,  4172,   561,   423,   284, 36535,   511,
          2057,   290,   550,  1310,  6946,   287,   511,  5682,    13,   198,
            43,  2821,  4740,   588, 11311,   290,  8234,   547,   845,  2408,
           284,  1064,   290,  4172,   550,   284,  1663,   511,   898,  2057,
           284,  7866,    13,   198, 10723,  5780,   338,   989,   284,  1064,
           503,   517,   492,   198, 21017]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [9]:
sample = model.generate(**inputs, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256)

/nethome/hkang342/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/nethome/hkang342/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [12]:
output = tokenizer.decode(sample[0])
print(output.strip().replace("Assistant:", ""))

#


In [11]:
print(output)

### Human: Please summarize the following article.

Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, families would have to ration their food and had little communication in their homes.
Luxuries like chocolate and fruit were very difficult to find and families had to grow their own food to survive.
Watch Martin's report to find out more..
### Assistant: The article is about how families during World War 2 had to ration their food and grow their own food to survive, as luxuries like chocolate and fruit were difficult to find. The reporter, Martin, explains the challenges of life during the war and how families had to adapt to survive.
<|endoftext|>
